In [ ]:
# Code to create a model used to predict whether the individual patient has an allergy
# Use different notebook to load the model and return a prediction

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow
import keras

import numpy as np
np.random.seed(123)

In [2]:
import matplotlib
from keras.models import * 
from keras.layers import *
from keras.optimizers import RMSprop
import pandas as pd

# Import both datasets, change to local path when running
patients = pd.read_excel(r"C:\Users\me\OneDrive\Desktop\Senior Design\Allergy_SanFrancisco\PATIENTS_Nov_3_2023_V4_sfm-data.xlsx", sheet_name="Level2_AI_Patient Traits")

allergies = pd.read_excel(r"C:\Users\me\OneDrive\Desktop\Senior Design\Allergy_SanFrancisco\PATIENTS_Nov_3_2023_V4_sfm-data.xlsx", sheet_name="Level1_Patient Allergens")

In [3]:
# Confirm that patient sheet imported correctly
# Comment below line before committing
# patients['SkinConditions'].head()

In [4]:
# Confirm allergy sheet imported correctly
# Comment line before committing
# allergies.head()

In [5]:
# Merge columns by ID if needed
patientAllergies = patients.merge(allergies, on = "SFM Id")
# Comment line before committing
# patientAllergies.head()

In [6]:
# Drop ID and location columns from dataframe
patientsTrimmed = patients.drop(['SFM Id', 'City', 'State', 'Country'], axis = 1)

In [7]:
# One-hot encode Gender column
patientsTrimmed['Gender'] = pd.Categorical(patientsTrimmed['Gender'].str.strip())
gender_onehot = pd.get_dummies(patientsTrimmed['Gender'], prefix = "Gender",
                                    prefix_sep = "-", dtype = int)
patientsTrimmed = patientsTrimmed.drop('Gender', axis = 1)
patientsTrimmed = patientsTrimmed.join(gender_onehot)

In [8]:
# One-hot encode SkinTone column
patientsTrimmed['SkinTone'] = pd.Categorical(patientsTrimmed['SkinTone'].str.strip())
skintone_onehot = pd.get_dummies(patientsTrimmed['SkinTone'], prefix = "SkinTone",
                                    prefix_sep = "-", dtype = int)
patientsTrimmed = patientsTrimmed.drop('SkinTone', axis = 1)
patientsTrimmed = patientsTrimmed.join(skintone_onehot)

In [9]:
# One-hot encode FitzPatrickSkinPhotoType column
patientsTrimmed['FitzpatrickSkinPhotoType'] = pd.Categorical(patientsTrimmed['FitzpatrickSkinPhotoType'].str.strip())
# Dropping first here since it is a blank variable in the column
fitzpatrick_onehot = pd.get_dummies(patientsTrimmed['FitzpatrickSkinPhotoType'], prefix = "Fitzpatrick",
                                    prefix_sep = "-", drop_first = True, dtype = int)
patientsTrimmed = patientsTrimmed.drop('FitzpatrickSkinPhotoType', axis = 1)
patientsTrimmed = patientsTrimmed.join(fitzpatrick_onehot)

In [ ]:
# Switching to TextVectorization (Tokenizer is deprecated)
from keras.layers import TextVectorization
# Replace commas with whitespace
patientsTrimmed['SkinConditions'] = patientsTrimmed['SkinConditions'].str.replace(',', ' ')
# Set the max length based on whitespace characters
max_len = patientsTrimmed['SkinConditions'].str.count(' ').max()
# Create TextVectorization object, separating on whitespace and using the max_len from earlier
vectorizer = TextVectorization(split = 'whitespace', output_sequence_length = max_len)

In [ ]:
# Adapt using the column we want to convert
vectorizer.adapt(patientsTrimmed['SkinConditions'].values)
# Reset the column after converting values to vector and placing in array
skinConditions = vectorizer(patientsTrimmed['SkinConditions']).numpy()
patientsTrimmed = patientsTrimmed.drop('SkinConditions', axis = 1)

In [12]:
patientsArr = patientsTrimmed.values
input_data = np.concatenate((patientsArr, skinConditions), axis = 1)

In [ ]:
''' Old code for SkinConditions conversion - leaving in case we need later
# Code to preprocess the SkinConditions column, tokenizing should be more recognizable to keras than regular text
## Delete if this idea does not work
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Flatten entries into actual lists
patientsTrimmed['SkinConditions'] = patientsTrimmed['SkinConditions'].apply(lambda x: ' '.join(x))
# Tokenize the characters of the columns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patientsTrimmed['SkinConditions'])
# Create and pad sequences of the tokens
patientsTrimmed['SkinConditions'] = tokenizer.texts_to_sequences(patientsTrimmed['SkinConditions'])
max_length = max(len(seq) for seq in patientsTrimmed['SkinConditions'])
testVar = pad_sequences(patientsTrimmed['SkinConditions'], maxlen = max_length, padding = "post")
patientsTrimmed['SkinConditions'] = testVar.tolist()
'''

In [14]:
# Drop ID column for preprocessing - ID should have no effect on prediction
allergiesNoId = allergies.drop('SFM Id', axis = 1)
# Remove all non-digit characters, then replace empty cells with NaN
allergiesNoId = allergiesNoId.replace(r'\D+', '', regex = True).replace('', np.nan)
# Set all NaN cells to 0
allergiesNoId = allergiesNoId.fillna(0)
# Convert entire dataframe to integer
allergiesNoId = allergiesNoId.astype(int)

In [15]:
allergiesNoId['AllergiesList'] = allergiesNoId.astype(str).apply(' '.join, axis=1)
allergiesNoId['AllergiesList'] = allergiesNoId['AllergiesList'].str.split()

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
# Create MultiLabelBinarizer object
mlb = MultiLabelBinarizer()

allergiesArr = np.array(allergiesNoId['AllergiesList'])
# Multi-hot encode data
allergiesArray = mlb.fit_transform(allergiesArr)

In [17]:
allergiesNew = mlb.inverse_transform(allergiesArray)

In [ ]:
allergiesDF = pd.DataFrame(mlb.transform(allergiesArr), columns = mlb.classes_)
allergiesDF.dtypes

In [104]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix
from keras.layers import Average

cv = KFold(n_splits=5, shuffle=True, random_state=7869)

fold_count = 1
# Train the model for each split
# Define the model inside the for loop
for train, test in cv.split(input_data,allergiesArray):

    n_classes = 731

    # Input layer
    input_shape = (40,)
    inputs = keras.Input(input_shape)
    # Hidden layers
    x1 = Dense(256, activation = 'sigmoid')(inputs)
    x1 = Dense(512, activation = 'sigmoid')(x1)
    # Output layer - use multilabel classification
    predictions1 = Dense(n_classes, activation='sigmoid')(x1)
    
    model1 = keras.Model(inputs = inputs, outputs = predictions1)
    model1.compile(loss=keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha = 0.35, gamma = 14), optimizer=keras.optimizers.Adam(learning_rate=0.1))
    
    x2 = Dense(256, activation = 'sigmoid')(inputs)
    # Output layer - use multilabel classification
    predictions2 = Dense(n_classes, activation='sigmoid')(x2)
    
    model2 = keras.Model(inputs = inputs, outputs = predictions2)
    model2.compile(loss=keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha = 0.1, gamma = 10), optimizer=keras.optimizers.SGD(learning_rate=0.05))
    
    x3 = Dense(385, activation = 'sigmoid')(inputs)
    # Output layer - use multilabel classification
    predictions3 = Dense(n_classes, activation='sigmoid')(x3)
    
    model3 = keras.Model(inputs = inputs, outputs = predictions3)
    model3.compile(loss=keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha = 0.5, gamma = .2), optimizer=keras.optimizers.Adam(learning_rate=0.01))
    
    x4 = Dense(256, activation = 'sigmoid')(inputs)
    x4 = Dense(512, activation = 'sigmoid')(x4)
    x4 = Dense(1024, activation = 'sigmoid')(x4)
    # Output layer - use multilabel classification
    predictions4 = Dense(n_classes, activation='sigmoid')(x4)
    
    model4 = keras.Model(inputs = inputs, outputs = predictions4)
    model4.compile(loss=keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha = 0.2, gamma = 5), optimizer=keras.optimizers.Adam(learning_rate=0.2))
    
    print("Running fold #: ", fold_count)

    fold_train_x =input_data[train]
    
    history1 = model1.fit(
    fold_train_x, allergiesArray[train],
    epochs = 20, 
    verbose = 0, 
    batch_size = 20, 
    validation_split=0.1
    )
    
    history2 = model1.fit(
    fold_train_x, allergiesArray[train],
    epochs = 10, 
    verbose = 0, 
    batch_size = 4, 
    validation_split=0.3
    )
        
    history3 = model1.fit(
    fold_train_x, allergiesArray[train],
    epochs = 50, 
    verbose = 0, 
    batch_size = 50, 
    validation_split=0.1
    )
            
    history4 = model1.fit(
    fold_train_x, allergiesArray[train],
    epochs = 15, 
    verbose = 0, 
    batch_size = 10, 
    validation_split=0.2
    )

    fold_test_x = input_data[test]
    y_true = allergiesArray[test]
    
    # pred_y = model.predict_classes(fold_test_x, verbose = 1)
    probs1 = model1.predict(fold_test_x, verbose = 0)
    probs2 = model2.predict(fold_test_x, verbose = 0)
    probs3 = model3.predict(fold_test_x, verbose = 0)
    probs4 = model4.predict(fold_test_x, verbose = 0)
    
    probs = (probs1 + probs2 + probs3 + probs4) / 4.0
    threshold = 0.5
    probs1 = (probs1 > threshold).astype(int)
    probs2 = (probs2 > threshold).astype(int)
    probs3 = (probs3 > threshold).astype(int)
    probs4 = (probs4 > threshold).astype(int)
    probs = (probs > threshold).astype(int)
    
    # AUC (Area Under Curve): how well the model can classify into the classes (high numbers better)
    auc = keras.metrics.AUC(multi_label = True, num_labels = 731, from_logits = False)
    # Precision: how well model predicts target class (high numbers better)\
    prec = keras.metrics.Precision()
    # Recall: how many objects model can find (high numbers better)
    rec = keras.metrics.Recall()
    # F1Score: mean between precision and recall
    f1 = keras.metrics.F1Score(average = 'weighted')

    auc.update_state(y_true, probs1)
    print("1 AUC: ", auc.result().numpy())
    prec.update_state(y_true, probs1)
    print("1 prec: ", prec.result().numpy())
    rec.update_state(y_true, probs1)
    print("1 rec: ", rec.result().numpy())
    f1.update_state(y_true, probs1)
    print("1 f1: ", f1.result().numpy())
    
    # AUC (Area Under Curve): how well the model can classify into the classes (high numbers better)
    auc = keras.metrics.AUC(multi_label = True, num_labels = 731, from_logits = False)
    # Precision: how well model predicts target class (high numbers better)\
    prec = keras.metrics.Precision()
    # Recall: how many objects model can find (high numbers better)
    rec = keras.metrics.Recall()
    # F1Score: mean between precision and recall
    f1 = keras.metrics.F1Score(average = 'weighted')

    auc.update_state(y_true, probs2)
    print("2 AUC: ", auc.result().numpy())
    prec.update_state(y_true, probs2)
    print("2 prec: ", prec.result().numpy())
    rec.update_state(y_true, probs2)
    print("2 rec: ", rec.result().numpy())
    f1.update_state(y_true, probs2)
    print("2 f1: ", f1.result().numpy())
    
    # AUC (Area Under Curve): how well the model can classify into the classes (high numbers better)
    auc = keras.metrics.AUC(multi_label = True, num_labels = 731, from_logits = False)
    # Precision: how well model predicts target class (high numbers better)
    prec = keras.metrics.Precision()
    # Recall: how many objects model can find (high numbers better)
    rec = keras.metrics.Recall()
    # F1Score: mean between precision and recall
    f1 = keras.metrics.F1Score(average = 'weighted')

    auc.update_state(y_true, probs3)
    print("3 AUC: ", auc.result().numpy())
    prec.update_state(y_true, probs3)
    print("3 prec: ", prec.result().numpy())
    rec.update_state(y_true, probs3)
    print("3 rec: ", rec.result().numpy())
    f1.update_state(y_true, probs3)
    print("3 f1: ", f1.result().numpy())
    
    # AUC (Area Under Curve): how well the model can classify into the classes (high numbers better)
    auc = keras.metrics.AUC(multi_label = True, num_labels = 731, from_logits = False)
    # Precision: how well model predicts target class (high numbers better)\
    prec = keras.metrics.Precision()
    # Recall: how many objects model can find (high numbers better)
    rec = keras.metrics.Recall()
    # F1Score: mean between precision and recall
    f1 = keras.metrics.F1Score(average = 'weighted')

    auc.update_state(y_true, probs4)
    print("4 AUC: ", auc.result().numpy())
    prec.update_state(y_true, probs4)
    print("4 prec: ", prec.result().numpy())
    rec.update_state(y_true, probs4)
    print("4 rec: ", rec.result().numpy())
    f1.update_state(y_true, probs4)
    print("4 f1: ", f1.result().numpy())
    
    
    # AUC (Area Under Curve): how well the model can classify into the classes (high numbers better)
    auc = keras.metrics.AUC(multi_label = True, num_labels = 731, from_logits = False)
    # Precision: how well model predicts target class (high numbers better)\
    prec = keras.metrics.Precision()
    # Recall: how many objects model can find (high numbers better)
    rec = keras.metrics.Recall()
    # F1Score: mean between precision and recall
    f1 = keras.metrics.F1Score(average = 'weighted')

    # print("Predicted y values: ", pred_y[0])
    print("X values for test: ", fold_test_x[0])
    yList = mlb.inverse_transform(y_true)
    print("Actual y values as list: ", yList[0])
    probsList = mlb.inverse_transform(probs)
    print("Predicted values as list: ", probsList[0])
    print(probs[0])
    
    auc.update_state(y_true, probs)
    print("Mean AUC: ", auc.result().numpy())
    prec.update_state(y_true, probs)
    print("Mean prec: ", prec.result().numpy())
    rec.update_state(y_true, probs)
    print("Mean rec: ", rec.result().numpy())
    f1.update_state(y_true, probs)
    print("Mean f1: ", f1.result().numpy())
    
    matrix = multilabel_confusion_matrix(y_true, probs)
    print(matrix)
    
    # print("Accuracy score sklearn: ", accuracy_score(allergiesArray[test], probs))
    accuracy = (allergiesArray[test] == probs).all(axis=(0,1)).mean()
    # print("Accuracy score equation: ", accuracy)

    fold_count = fold_count + 1

Running fold #:  1
1 AUC:  0.3488372
1 prec:  0.5121212
1 rec:  0.13526313
1 f1:  0.13221318
2 AUC:  0.34968635
2 prec:  0.007648342
2 rec:  0.35790035
2 f1:  0.039280396
3 AUC:  0.3487258
3 prec:  0.011826358
3 rec:  0.5575268
3 f1:  0.18649349
4 AUC:  0.3488437
4 prec:  0.008818079
4 rec:  0.4169946
4 f1:  0.04468945
X values for test:  [1950    1    0    0    0    0    0    1    0    0    0    0    0    0
    0    0    0    3    5    2    6    4    8    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
Actual y values as list:  ('0', '124', '144962', '1462', '1590', '1600', '1607', '1827', '188', '5468', '5986', '9300')
Predicted values as list:  ('0', '104', '1153', '11661', '121146', '12522', '130118', '136703', '148092', '1496', '1589', '1590', '1592', '1597', '1602', '1610', '1826', '183', '1903', '192', '1997', '2013', '2023', '2044', '215', '24061', '257', '2853', '28559', '30442', '33173', '3799', '38080', '41729', '41989', '4550', '4617', '4

Running fold #:  4
1 AUC:  0.34336525
1 prec:  0.25555837
1 rec:  0.1408382
1 f1:  0.1378429
2 AUC:  0.34418455
2 prec:  0.011345779
2 rec:  0.56495404
2 f1:  0.18066192
3 AUC:  0.34278378
3 prec:  0.009244991
3 rec:  0.46776664
3 f1:  0.07648648
4 AUC:  0.34337705
4 prec:  0.011249365
4 rec:  0.5951685
4 f1:  0.18961269
X values for test:  [1985    1    0    0    0    0    0    1    0    0    0    0    0    0
    0    0    0    3    7    5    2    4    8    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
Actual y values as list:  ('0', '12522', '1598', '1605', '1805', '201', '2023', '215', '2350', '257', '263', '3799', '5236', '5467', '9315')
Predicted values as list:  ('0', '103637', '104630', '105017', '119', '11996', '124', '12522', '128', '13018', '131996', '132762', '133', '136', '1496', '1582', '1586', '1590', '1596', '1597', '1599', '1600', '1601', '1605', '1610', '1613', '1635', '1802', '1826', '183', '184', '185', '188', '1903', '2023', '20

In [ ]:
# TODO: Save Keras model as separate file